In [4]:
from tensorflow.python.client import device_lib
def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 
# my output was => ['/device:CPU:0']
# good output must be => ['/device:CPU:0', '/device:GPU:0']


['/device:CPU:0', '/device:GPU:0']


2021-11-28 03:42:32.449268: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-28 03:42:32.478853: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 03:42:32.502866: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-28 03:42:32.502996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zer

In [56]:
import collections
import pathlib
import pandas as pd
import tensorflow as tf

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import utils
from tensorflow.keras.layers import TextVectorization

import tensorflow_datasets as tfds
import tensorflow_text as f_text



In [71]:
import re

def preprocess(message):
    """

        - lowercase
        - remove URLs
        - removes punctuation
        - removes any single character tokens
        - removes emojies
        """
    # Lowercase the twit message
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    text = message.lower()
    test = emoji_pattern.sub(r' ', text)
    # Replace URLs with a space in the message
    text = re.sub('https?:\/\/[a-zA-Z0-9@:%._\/+~#=?&;-]*', ' ', text)
    # Replace everything not a letter or apostrophe with a space
    text = re.sub('[^a-zA-Z\']', ' ', text)
    # Remove single letter words
    text = ' '.join( [w for w in text.split() if len(w)>1] )
    return text
        

In [72]:
import csv
csv_file = 'train.csv'
messages = []
subreddits = []
with open(csv_file, 'r') as f:
    reader = csv.DictReader(f)
    for row in reader:
        messages.append(row.get('text'))
        subreddits.append(row.get('subreddit'))

In [73]:
preprocessed = [preprocess(message) for message in messages]

In [74]:
setofsubs = set(subreddits)
print(setofsubs)
# convert all different sub values into integer
def subreddittonum(shortsub):
    return {
            'worldnews': 0,
            'Jokes': 1,
            'antiwork': 2,
            'SubredditSimulator': 3,
            'wallstreetbets': 4,

    }[shortsub]


{'Jokes', 'SubredditSimulator', 'antiwork', 'wallstreetbets', 'worldnews'}


In [75]:
subreddits = [subreddittonum(subreddit) for subreddit in subreddits]

In [76]:
from sklearn.model_selection import train_test_split
dataset, info = preprocessed, subreddits
dataset_train, dataset_test, info_train, info_test = train_test_split(dataset, info, test_size=0.05)
#we use one_hot because we are trying to classify for 5 different classes. so we create an array. A smarter programmer would do this in the text to num function but im not smarter. 
train_dataset = tf.data.Dataset.from_tensor_slices((dataset_train, tf.one_hot(info_train, depth=len(setofsubs))))
test_dataset = tf.data.Dataset.from_tensor_slices((dataset_test, tf.one_hot(info_test, depth=len(setofsubs))))

In [77]:
BUFFER_SIZE = 500
#smol epoch to 2070 super doesnt cry.
BATCH_SIZE = 8


In [78]:
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [79]:
VOCAB_SIZE = 6900
encoder = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))


In [84]:

model_double = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(124,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(124)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(5)
])


In [85]:
model_double.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
model_old = model_double #0.8667
history_double = model_double.fit(train_dataset, epochs=4,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/4
3488/3488 [==============================] - 145s 39ms/step - loss: 0.2347 - accuracy: 0.7340 - val_loss: 0.1541 - val_accuracy: 0.8417
Epoch 2/4
3488/3488 [==============================] - 116s 33ms/step - loss: 0.1271 - accuracy: 0.8765 - val_loss: 0.1575 - val_accuracy: 0.8500
Epoch 3/4
2221/3488 [==================>...........] - ETA: 45s - loss: 0.0961 - accuracy: 0.9094

In [29]:
def numtosub(shortsub):
    return {
            0:'worldnews',
            1:'Jokes',
            2:'antiwork',
            3:'SubredditSimulator',
            4:'wallstreetbets',

    }[shortsub]

In [33]:
import csv
csv_file = 'test_text.csv'
messages_test = []
subreddits_test = []
iteration = 0
with open(csv_file, 'r') as ayylmao:
    reader = csv.DictReader(ayylmao)
    for row in reader:
        iteration = iteration+1
        message = preprocess(row.get('text'))
        messages_test.append(message)
        try:
            sub = numtosub(np.argmax(model_old.predict(np.array([message]))))
        except:
            sub = "SubredditSimulator"
        subreddits_test.append(sub)

2021-11-28 04:39:46.305053: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at strided_slice_op.cc:108 : INVALID_ARGUMENT: slice index 0 of dimension 0 out of bounds.
2021-11-28 04:39:46.305078: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at strided_slice_op.cc:108 : INVALID_ARGUMENT: slice index 0 of dimension 0 out of bounds.
2021-11-28 04:41:19.312733: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at strided_slice_op.cc:108 : INVALID_ARGUMENT: slice index 0 of dimension 0 out of bounds.
2021-11-28 04:41:19.312758: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at strided_slice_op.cc:108 : INVALID_ARGUMENT: slice index 0 of dimension 0 out of bounds.
2021-11-28 04:42:03.590148: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at strided_slice_op.cc:108 : INVALID_ARGUMENT: slice index 0 of dimension 0 out of bounds.
2021-11-28 04:42:03.590169: W tensorflow/core/framework/op_kernel.cc:1745] 

In [34]:
#save to csv for upload
import pandas as pd
df = pd.read_csv("test_text.csv")
df["id"] = df.reset_index().index
df["subreddit"] = subreddits_test
df.to_csv("final_final.csv", index=False)